In [1]:
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()


/home/sandhya/anaconda3/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [4]:
%run ../paths.py
CONST = CONST()

In [586]:
train = pd.read_csv(CONST.TRAIN_DATA)
train.head()

,Unnamed: 0,tweet,class
0,12761,Mannnnnn these hoes aint loyal,1
1,25141,have a fabulous weekend beautiful souls #nofil...,0
2,53167,#fathersday #dad @user,0
3,17294,RT @SheswantstheD: these hoes ain't loyal http...,1
4,29359,newhairâ¨ð #lapis#rollen#new#hair#brond#gr...,0


In [245]:
train_copy = train.copy()
train_copy.head()

,Unnamed: 0,tweet,class
0,12761,Mannnnnn these hoes aint loyal,1
1,25141,have a fabulous weekend beautiful souls #nofil...,0
2,53167,#fathersday #dad @user,0
3,17294,RT @SheswantstheD: these hoes ain't loyal http...,1
4,29359,newhairâ¨ð #lapis#rollen#new#hair#brond#gr...,0


In [5]:
%run ../src/processing/glove_utils.py

In [6]:
glove_embed = load_embed(CONST.GLOVE_100d)

In [592]:
'<user>' in glove_embed, '<number>' in glove_embed, '<c>' in glove_embed, '<repeat>' in glove_embed,"<allcaps>" in glove_embed

(True, True, False, True, True)

In [312]:
vocab = build_vocab(list(train_copy['tweet'].apply(lambda x:x.split())))
vocab
oov = check_coverage(vocab,glove_embed)
#oov[:10]

100%|██████████| 97055/97055 [00:00<00:00, 228926.20it/s]


Found embeddings for 20.63% of vocab
Found embeddings for  67.83% of all text


In [438]:
add_lower(glove_embed, vocab)

Added 0 words to embedding


In [441]:
#glove_embed['us']

In [315]:
%run ../src/processing/html_tag_remover.py

In [316]:
train_copy['tweet'] = train_copy['tweet'].apply(lambda x : strip_tags(x))

In [337]:
%run ../src/processing/regular_expressions.py

 joke: what do you call a pig with three eyes? <repeat> piiig! <repeat> allcaps <allcaps>  . <repeat>  <number> <number> 


In [338]:
train_copy['tweet'] = train_copy['tweet'].apply(lambda x: tweet_preprocessing(x))


In [320]:
vocab = build_vocab(list(train_copy['tweet'].apply(lambda x:x.split())))
#vocab
oov = check_coverage(vocab,glove_embed)

100%|██████████| 65444/65444 [00:00<00:00, 226897.17it/s]


Found embeddings for 41.40% of vocab
Found embeddings for  86.78% of all text


In [322]:
oov[:10]
#%run ../src/processing/contractions.py

[("i'm", 1946),
 ("don't", 1635),
 ('“', 1234),
 ("can't", 982),
 ("ain't", 712),
 ("you're", 651),
 ('bihday', 584),
 ('bitch.', 509),
 ("y'all", 353),
 ("i'll", 270)]

In [324]:
train_copy['tweet'] = train_copy['tweet'].apply(lambda x: clean_contractions(x, contraction_mapping))
#train_copy.head()


In [325]:
vocab = build_vocab(list(train_copy['tweet'].apply(lambda x:x.split())))
#vocab
oov = check_coverage(vocab,glove_embed)

100%|██████████| 65351/65351 [00:00<00:00, 157328.05it/s]

Found embeddings for 41.44% of vocab
Found embeddings for  88.38% of all text


In [326]:
oov[:10]

[('“', 1234),
 ('bihday', 584),
 ('bitch.', 509),
 ('😂', 224),
 ('day!', 222),
 ('bitch,', 200),
 ('…', 193),
 ('<url>”', 163),
 ('trash.', 155),
 ('😂😂', 145)]

In [263]:
import re
def ap(text):
    text = text.group()
    return text.replace("'","")
re.sub("\w+[']s",ap,"'s  'father's,mother's,brother's,sister's,baby's father's also uncle")

"'s  'fathers,mothers,brothers,sisters,babys fathers also uncle"

In [327]:
%run ../src/processing/spellings.py

In [328]:
train_copy['tweet'] = train_copy['tweet'].apply(lambda x: replace_elongated_text(x))


In [329]:
vocab = build_vocab(list(train_copy['tweet'].apply(lambda x:x.split())))
#vocab
oov = check_coverage(vocab,glove_embed)

100%|██████████| 64895/64895 [00:00<00:00, 168517.12it/s]


Found embeddings for 41.80% of vocab
Found embeddings for  88.46% of all text


In [330]:
oov[:10]

[('“', 1234),
 ('bihday', 584),
 ('bitch.', 509),
 ('😂😂', 379),
 ('😂', 224),
 ('day!', 222),
 ('bitch,', 200),
 ('…', 193),
 ('<url>”', 163),
 ('trash.', 155)]

In [331]:
%run ../src/processing/special_char.py

In [333]:
train_copy['tweet'] = train_copy['tweet'].apply(lambda x: clean_special_chars(x,punct,punct_mapping))


In [339]:
vocab = build_vocab(list(train_copy['tweet'].apply(lambda x:x.split())))
#vocab
oov = check_coverage(vocab,glove_embed)

100%|██████████| 47082/47082 [00:00<00:00, 196901.27it/s]


Found embeddings for 62.03% of vocab
Found embeddings for  96.45% of all text


In [340]:
oov[:10]

[('bihday', 636),
 ('😂😂', 447),
 ('😂', 270),
 ('â\x9d¤ï¸\x8f', 122),
 ('★', 121),
 ('ð\x9f\x98\x8a', 116),
 ('ð\x9f\x98\x8d', 103),
 ('ð\x9f\x98', 86),
 ('😒', 82),
 ('ð\x9f\x98\x82', 73)]

In [341]:
%run ../src/processing/emojis.py

smiling_face fmgvmn steaming_bowl  sleeping_face  exploding_head  woozy_face fgfb thinking_face


In [398]:
#train_copy['removed_contr'][len('removed_contr')<0]
#train_copy.head()
drop_rows = train_copy[train_copy['tweet']==""].tweet
train_copy.drop(drop_rows.index, axis=0,inplace=True)
#train_copy.drop([[1859]],inplace=True)
train_copy['tweet'] = train_copy['tweet'].apply(lambda x: remove_duplicates(x) )


In [399]:
vocab = build_vocab(list(train_copy['tweet'].apply(lambda x:x.split())))
#vocab
oov = check_coverage(vocab,glove_embed)


100%|██████████| 46972/46972 [00:00<00:00, 192029.38it/s]

Found embeddings for 62.18% of vocab
Found embeddings for  96.45% of all text


In [400]:
oov[:10]

[('😂', 717),
 ('bihday', 636),
 ('â\x9d¤ï¸\x8f', 122),
 ('★', 121),
 ('ð\x9f\x98\x8a', 116),
 ('ð\x9f\x98\x8d', 103),
 ('😒', 90),
 ('😭', 88),
 ('ð\x9f\x98', 86),
 ('ð\x9f\x98\x82', 73)]

In [401]:
import string
white_list = string.ascii_letters + string.digits + ' '
white_list += "'"
#oov[:100]

In [402]:
glove_chars = ''.join([c for c in tqdm(glove_embed) if len(c) == 1])
glove_symbols = ''.join([c for c in glove_chars if not c in white_list])
glove_symbols


100%|██████████| 1193534/1193534 [00:02<00:00, 502976.78it/s]


'.:,!"?()-/*>^<&_~;|[]`$=+%@\\#}{'

In [456]:
all_characters = build_vocab(list(train_copy["tweet"]))
all_characters
#symbols_to_delete = ''.join([c for c in all_characters if not c in white_list if not c in glove_symbols])
#symbols_to_delete
del all_characters["<"]
del all_characters[">"]
all_characters
symbols_to_delete

100%|██████████| 42348/42348 [00:01<00:00, 23162.40it/s]


'ð\x9f\x98â\x9c¨\x92\x87\x80¦👏\x86\x9d\x93\x99\x8f\x8e\x89¤ï¸😩😂¬\x8a\x90¶¾😭\x9e\x95😅💊👫❤½✌️★\x8d\x8b»\x84\x8c¹👋\x96😳\xad\x82©®👀😏ó\x94😋😌¼✂😻🙌💦😎👌🙅✋🚫🍞💁😪ª\x9a\x91«🙏\x81ã👟\x9b\x88\x97💯¢¥😷¡\x83😕😍😒º💀😢😴\U000feb9f¿👊💸😖🙈¯💩ìíè🆗🆒äçåµñ⊕🙊👅🎵ü😈🍻🎉🎈😊🔫☺👬👭💥§😆♥◡̈⃝😉😜é😫🐸☕😁😱‼💙💃🎤👐🌈🎭😘💍💕🔐✊💋🔪±😹\ue412🐛💰·✈�ëê😐😡👉👈🔴⚫👎😑💜❗👼😔æ→👩👆👯😝🐎\U000feb97\ue411\ue427🏆📝📃🏀🐼💅😤📑😠🐱😶🍟á😃📖û👑👿\U000fe343😥🍇🍼😛🚮💖😺✨🌟💫😞🎂🍰🍑💑💆😣🎄⛄🎅🙋🍁\U000fe35b\U000feb5d\U000fe358\U000feb7b\U000fe334\U000fec11\U000feb9d💎👍💵🚨💧🐢😦🙇🌵👸💞🔥💨🎼🎁🌴☀🙎\U000feb9e💄💗🔑👶🚼☝🎲⃣🐥🐣😾。🚌🙆💏🚶\U000fe32c🎶🎧🙉😿øù💡🍅😟🐷📰😓🐊💪😮🇺🇸\U000fe326❓⭐🍍ö😵🐦🐝🐯🍃🌿\ue31f\ue301📕📚🏡😬🙀ò💤😨👄🎬🎥\U000feba0💛🌙☁🌠🆚👻🏈😗😄\U000fe346\U000feb0f💚🎊\U000fe33a⚡👮💲📢⛽⚾🍂🚙\U000fe327👇🚕⚪\ue40d\ue04f\ue40c\ue107🐙🍄✔💣🍆😲🕛′🍗♿🇬🇧😧🏄🍪❄🌀📣👰\ue108👧💢🇮🇹✖\U000feb5b\U000fe321🍺👷🌝🏃🎯👂🐬🐐\U000fe194\U000fe355❌⭕🌹👞🙍˘🐤👠👛💳🍉😽⬇🏂😰👺🚧🗿🌊🏊😇î♨🔊🍕ú♫\U000fe511\U000fe4e4\U000fe4dd🌾🔚\ue41f🐠❕🚬🍸\U000fe190♡❔\U000fe4f5🌸🚓🚒💺🍖🍝🍤🍛📷💔👹😚\U000fe320💓💇👵💉🐈🐶😼🚀⌚💂💭🍫🐒\U000fe351\U000fe335😯👲🚘🚩🌽💘📋\U000fe340🌳\U000fe825\U000feb99\U000fe347🍵🚥🏰\ue40e\ue24d\ue337👽🔓\U000feb5c🅰🐺📼🐍łę⏳🐂，\ue420🍩👙📉🌚'

In [448]:
remove_dict = {ord(c):f'' for c in symbols_to_delete}
isolate_dict = {ord(c):f' {c} ' for c in symbols_to_isolate}

def handle_punctuation(x):
    x = x.translate(remove_dict)
    x = x.translate(isolate_dict)
    return x


In [449]:
train_copy['tweet'] = train_copy['tweet'].apply(lambda x: handle_punctuation(x))


In [450]:
vocab = build_vocab(list(train_copy['tweet'].apply(lambda x:x.split())))
#vocab
oov = check_coverage(vocab,glove_embed)

100%|██████████| 40761/40761 [00:00<00:00, 186036.52it/s]

Found embeddings for 72.19% of vocab
Found embeddings for  98.05% of all text


In [455]:
"!!" in vocab

False

In [447]:
symbols_to_isolate = ''.join([c for c in all_characters  if c not in white_list if c in glove_symbols])
symbols_to_isolate

':#.,!-"/?&~*=@$;%+(|)^{}\\[]'

In [460]:
"ASAP" in glove_embed

False

In [7]:
df = pd.read_csv(CONST.TRAIN_DATA)
df.head()
df_copy1 = df.copy()

In [8]:
%run ../src/processing/regular_expressions.py
%run ../src/processing/special_char.py
%run ../src/processing/cleaning.py

 what do you call a pig with three eyes? <repeat> piiig! <repeat> allcaps <allcaps>  . <repeat> [dbr <number> !@ <number> <number> 
dfvfe ffve rfvfe sdv eee
dfvfe ffve rfvfe sdv eee
talk to you later wait
 what do you call a pig with three eyes? <repeat> piiig! <repeat> allcaps <allcaps>  . <repeat> [dbr <number> !@ <number> <number> 


In [18]:
d=clean_data(df_copy1)

In [19]:
drop_rows = d[d['tweet']==""]
d.drop(drop_rows.index, axis=0,inplace=True)
vocab = build_vocab(list(d['tweet'].apply(lambda x:x.split())))
#vocab
oov = check_coverage(vocab,glove_embed)

100%|██████████| 28732/28732 [00:00<00:00, 333001.95it/s]

Found embeddings for 83.58% of vocab
Found embeddings for  98.67% of all text


In [20]:
d.tweet.head()

0                  mann these hoes aint loyal
1    have a fabulous weekend beautiful souls 
2                                            
3                    these hoes is not loyal 
4                                  newhair l 
Name: tweet, dtype: object

In [21]:
#can use spellchecker to increase the coverage
oov[:100]

[('bihday', 451),
 ('staing', 54),
 ('erience', 53),
 ('impoant', 49),
 ('staed', 43),
 ('toptags', 27),
 ('ected', 23),
 ('unfounately', 20),
 ('suppoers', 19),
 ('suppoing', 19),
 ('ectations', 19),
 ('fuher', 17),
 ('ensive', 17),
 ('cultureofdevelopment', 17),
 ('ukhxint', 16),
 ("fuckin'", 16),
 ('osed', 15),
 ('nothe', 15),
 ('spos', 15),
 ('brexit', 13),
 ('muzzie', 13),
 ('aicle', 12),
 ('paner', 12),
 ('oppounity', 11),
 ('factsguide', 11),
 ('heabreaking', 10),
 ('ceain', 10),
 ('soed', 10),
 ('appletstag', 9),
 ('eriences', 9),
 ('lains', 9),
 ('livelypics', 9),
 ('woulday', 9),
 ("'i", 9),
 ('bihdays', 8),
 ('polarisation', 8),
 ('leftright', 8),
 ('wakeuppeopl', 8),
 ('summeime', 8),
 ("gon'", 8),
 ('yelchin', 8),
 ('ecting', 8),
 ('heabroken', 8),
 ('fleek', 8),
 ('kscrashcorrectors', 8),
 ('osing', 7),
 ('jivemap', 7),
 ('staer', 7),
 ('oppounities', 7),
 ('suppoer', 7),
 ('nohern', 7),
 ('altright', 7),
 ("day'", 7),
 ('giphy', 7),
 ('bihdaysway', 6),
 ('miscegenation',

In [481]:
"<user>" in glove_embed

True